# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
#### Prof. Mayron César O. Moreira (DCC/UFLA)
#### Aluno: Bruno Crespo Ferreira

## Problema

Implemente o algoritmo Branch and Bound para resolver o problema da mochila binária. Para resolver os subproblemas, implemente o algoritmo guloso para a mochila fracionária. A escolha dos nós para a expansão da árvore Branch and Bound deve ser feita pelo melhor limitante superior. Como limite inferior, implemente uma heurística gulosa. Em sua resposta, você deve apresentar:

* Número de nós explorados;
* GAP de otimalidade da solução encontrada;
* Tempo computacional gasto.

## Instância

Seu código deve ser testado na seguinte instância: [link](https://drive.google.com/file/d/12CeZEow-6vVgJFgzXMo0gbjV5hLCM6Zi/view?usp=sharing). O README se encontra em: [link](https://drive.google.com/file/d/1LBTdkDoTQRxUJsKLI4Z38-Uc8oUPCZQ0/view?usp=sharing).

## Modelagem Matemática

### Parâmetros

* $n$: Número de itens;
* $w_i$: Peso do item $i, i = \{1,...,n\}$;
* $v_i$: Valor do item $i, i = \{1,...,n\}$;
* C: Capacidade da mochila.

### Variáveis

* $x_i$: Variável binária que indica se o item $i$ foi selecionado $(x_i = 1)$ ou não $(x_i = 0)$.

### Função Objetivo

Consiste em **maximizar** o valor total dos itens na mochila.
\begin{equation}
max \sum_{i=1}^n v_{i} x_{i}
\end{equation}

### Restrições

* Restrição 1: Deve-se respeitar a capacidade da mochila.
\begin{equation}
\sum_{i=1}^n w_{i} x_{i} \le C
\end{equation}

* Restrição 2: Cada item pode ser selecionado ou não.
\begin{equation}
x_i \in \{0,1\}, \quad ∀i = \{1,...,n\}
\end{equation}

### Modelo

\begin{equation}
max \sum_{i=1}^n v_{i} x_{i}
\end{equation}

sujeito a:

\begin{alignat}{2}
\sum_{i=1}^n w_{i} x_{i} \le C \\
x_i \in \{0,1\}, \quad ∀i = \{1,...,n\}
\end{alignat}

## Resolução do Problema

### Importando bibliotecas

In [57]:
import time
import numpy as np

### Função para leitura do arquivo

In [58]:
def load_instance(filename):
  with open(filename, 'r') as f:
    lines = f.readlines()

  instance_name = lines[0].strip()
  instance_class = lines[1].strip()
  capacity = int(lines[2].strip())
  items = []

  for idx, line in enumerate(lines[3:]):
    weight, value = map(int, line.strip().split())
    items.append((idx, weight, value))

  return instance_name, instance_class, capacity, items

### Heurística gulosa para o limite inferior e Algoritmo guloso para a mochila fracionária

In [59]:
def fractional_knapsack(capacity, items):
  items = sorted(items, key=lambda x: x[2]/x[1], reverse=True)
  total_value = 0
  remaining_capacity = capacity

  for _, weight, value in items:
    if weight <= remaining_capacity:
      total_value += value
      remaining_capacity -= weight
    else:
      total_value += value * (remaining_capacity / weight)
      break

  return total_value

### Classe para o nó do Branch and Bound

In [60]:
class Node:
  def __init__(self, level, profit, weight, taken):
    self.level = level
    self.profit = profit
    self.weight = weight
    self.taken = taken

### Cálculo do melhor limite superior para escolher o próximo nó

In [61]:
def calculate_bound(node, capacity, items):
  if node.weight > capacity:
    return 0

  remaining_capacity = capacity - node.weight
  total_value = node.profit

  for i in range(node.level + 1, len(items)):
    _, weight, value = items[i]
    if weight <= remaining_capacity:
      total_value += value
      remaining_capacity -= weight
    else:
      total_value += value * (remaining_capacity / weight)
      break

  return total_value

### Algoritmo Branch and Bound

In [62]:
def branch_and_bound_knapsack(capacity, items):
  start_time = time.time()
  items = sorted(items, key=lambda x: x[2]/x[1], reverse=True)

  # Inicialização
  best_solution = 0
  best_taken = []
  root = Node(level=-1, profit=0, weight=0, taken=[])
  queue = [root]
  explored_nodes = 0

  # Processando os nós
  while queue:
    node = max(queue, key=lambda x: calculate_bound(x, capacity, items))
    queue.remove(node)
    explored_nodes += 1

    if node.level == len(items) - 1:
      continue

    # Expansão do nó
    next_level = node.level + 1
    _, weight, value = items[next_level]

    # Caso 1: Incluir o item
    if node.weight + weight <= capacity:
      taken = node.taken[:]
      taken.append(items[next_level][0])
      profit = node.profit + value
      new_node = Node(level=next_level, profit=profit, weight=node.weight + weight, taken=taken)

      if profit > best_solution:
        best_solution = profit
        best_taken = taken[:]

      if calculate_bound(new_node, capacity, items) > best_solution:
        queue.append(new_node)

    # Caso 2: Não incluir o item
    taken = node.taken[:]
    new_node = Node(level=next_level, profit=node.profit, weight=node.weight, taken=taken)

    if calculate_bound(new_node, capacity, items) > best_solution:
        queue.append(new_node)

  end_time = time.time()

  # Calcula-se o GAP de otimalidade
  fractional_solution = fractional_knapsack(capacity, items)
  gap = 100 * (fractional_solution - best_solution) / fractional_solution

  return best_solution, best_taken, explored_nodes, gap, end_time - start_time

### Imprimindo as soluções do problema

In [63]:
def main():
  filename = "Weakly001"
  instance_name, instance_class, capacity, items = load_instance(filename)

  best_value, best_taken, explored_nodes, gap, runtime = branch_and_bound_knapsack(capacity, items)

  total_weight = sum(items[item_id][1] for item_id in best_taken)
  remaining_capacity = capacity - total_weight

  print(f"Instância: {instance_name}")
  print(f"Classe: {instance_class}")
  print(f"Capacidade da mochila: {capacity}")
  print(f"Peso total selecionado: {total_weight}")
  print(f"Capacidade restante: {remaining_capacity}")
  print(f"Melhor valor encontrado: {best_value}")
  print(f"Itens selecionados: {best_taken}")
  print(f"Nós explorados: {explored_nodes}")
  print(f"GAP de otimalidade: {gap:.2f}%")
  print(f"Tempo de execução: {runtime:.4f} segundos")

main()

Instância: Weakly001
Classe: WeaklyCorrelated
Capacidade da mochila: 2453
Peso total selecionado: 2452
Capacidade restante: 1
Melhor valor encontrado: 4886
Itens selecionados: [293, 744, 0, 331, 369, 922, 590, 967, 328, 472, 988, 504, 102, 9, 12, 316, 792, 78, 295, 395, 110, 936, 618, 3, 989, 13, 631, 139, 930, 686, 557, 788, 978, 181, 993, 837, 841, 555, 886, 30, 218, 901, 689, 674, 470, 954, 496, 511, 573, 699, 751]
Nós explorados: 1050
GAP de otimalidade: 0.03%
Tempo de execução: 0.2506 segundos
